# Load packages

In [3]:
# Packages onto Conda\n",
!install.packages("shiny")
!install.packages("shinydashboard")
!install.packages("ggplot2")

also installing the dependencies ‘httpuv’, ‘later’

Warning message in install.packages("shiny"):
“installation of package ‘later’ had non-zero exit status”Warning message in install.packages("shiny"):
“installation of package ‘httpuv’ had non-zero exit status”Warning message in install.packages("shiny"):
“installation of package ‘shiny’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


ERROR: Error in !install.packages("shiny"): invalid argument type


In [13]:
# Packages onto jupyter\n",
install.packages('ggthemes')
install.packages("anchors")

# Load libraries",
library(shiny)
library(ggplot2)
library(ggthemes)
library(scales)
library(anchors)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


# Data Sim

In [17]:
# Load csvs
setwd("/Users/chloemaine/Documents/Chloe/BGSE/blockchain/final_project")

loss <- read.csv(file = 'evidence_loss_bystage.csv')
colnames(loss) <- c('Stage', 'PT', 'Number', 'X')

big_df <- read.csv(file = 'big_df.csv')

# Pie charts\n",
et_df <- data.frame(
  Evidence.Type = c("Documents", "Videos", "Obscene publications", "Weapons", "Drugs", "Counterfeits", "Photos", "Other"),
  value = c(9, 14, 8, 3, 27, 13, 7, 19)
  )

dt_df <- data.frame(
  Department = c("National Crime Agency", "Local Police", "Local Forensics Dpt", "Criminal lawyers", "Forensics experts", "Police", "Criminal Court", "Other"),
  value = c(9, 42, 26, 2, 5, 4, 6, 4)
  )

ct_df <- data.frame(
  Case.Type = c("Violent crime", "Vehicle crime", "Burglary", "Criminal damage &Arson", "Theft", "Possession of weapon", "Drugs", "Other"),
  value = c(32, 3, 16, 14, 22, 5, 7, 45)
  )

dataset <- big_df

# Define functions

In [18]:
blank_theme <- theme_minimal()+
      theme(
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
      panel.border = element_blank(),
      panel.grid=element_blank(),
      axis.ticks = element_blank(),
      plot.title=element_text(size=14, face="bold")
      )

# Shiny Dash

In [ ]:
####''' This is a Shiny web application. You can run the application by running this cell'''

# Define UI
ui <- navbarPage("Criminal Evidence Management",
                 inverse = TRUE,
 
                 # Tab One
                 tabPanel("Lost Evidence",

                          plotOutput('superPlot'),

                          hr(),

                          fluidRow(
                                   column(3,
                                       h4("Time filter"),
                                       sliderInput('Date_range', 'Days after January 1st', 
                                                   min=1, max=365, value=min(1000, nrow(dataset)), 
                                                   step=10, round=0),
                                    br(),
                                    checkboxInput('jitter', 'Jitter'),
                                    checkboxInput('smooth', 'Smooth'),
                                          ),
                                   column(4, offset = 1,
                                        selectInput('x', 'X', c("Monetary.Value","Impact.Loss", "Days.Til.Escalation", "Day.lost",  "Item.height", "Item.weight", "Item.length")),
                                        selectInput('y', 'Y', c("Evidence.Type", "Case.Type", "Department", "Days.Til.Escalation")),
                                        selectInput('color', 'Color', c("Case.Type",'None', "Evidence.Type",  "Department"))
                                          ),
                                   column(4,
                                        selectInput('facet_row', 'Facet Row', c(None='.', "Evidence.Type", "Case.Type", "Department")),
                                        selectInput('facet_col', 'Facet Column', c(None='.', "Evidence.Type", "Case.Type", "Department"))
                                         )
                                  )
                            ),  

                 # Tab Two
                 tabPanel("Loss by stage",
                          sidebarLayout(sidebarPanel(width = 3,
                                                     selectInput("Plot.Type", "Plot type",
                                                                  choices= list("Total",
                                                                                "Total loss",
                                                                               "Deliberate damage",
                                                                                "Other damage",
                                                                                "Theft")
                                                                )
                                                    ),
                                         mainPanel(plotOutput("lossPlot"))
                                       )
                          ),
    
                # Tab Three
                tabPanel("Loss by type",
                         fluidRow(column(width = 4, plotOutput("etPlot")),
                                 column(width = 4, plotOutput("dtPlot")),
                                 column(width = 4, plotOutput("ctPlot")),
                                 )
                         )
                 )

# Define server logic 
server <- function(input, output) {

    # Super plot


    output$superPlot <- renderPlot({
        
        dataset = subset(big_df, Day.lost <= input$Date_range)

        p <- ggplot(dataset, aes_string(x=input$x, y=input$y)) + geom_point() +
                ggtitle('Super Graph Creator: Explore all lost Criminal Evidence') +
                theme(title = element_text(size = 16, face = "bold"))

        if (input$color != 'None')
          p <- p + aes_string(color=input$color)

        facets <- paste(input$facet_row, '~', input$facet_col)
        if (facets != '. ~ .')
          p <- p + facet_grid(facets)

        if (input$jitter)
          p <- p + geom_jitter()
        if (input$smooth)
          p <- p + geom_smooth()

        print(p)

      })


    # Loss plot
    output$lossPlot <- renderPlot({

        ggplot(subset(loss, PT == input$Plot.Type, select = c(Number, Stage)),
           aes(Stage, Number)) +
        geom_col() + 
        xlab("Stage") + 
        ylab("Number of Items")    +
        ggtitle(input$Plot.Type, 'Evidence Items by Stage') + 
        theme_economist() + 
        scale_colour_economist() + 
        scale_x_discrete(name ="Stage", 
                    limits=c('Incident Reponse',
                             'Evidence Collection',
                             'Transportation',
                             'Forensic Investigation',
                             'Storage',
                             'Additional Forensics',
                             'Transfer to court',
                             'Archiving')) +
        theme(axis.text.x = element_text( size=12, angle=20),
              axis.title.x = element_text(size = 12, face = "bold"),
              axis.title.y = element_text(size = 12, face = "bold"),
              title = element_text(size = 16, face = "bold"))

    })

    # Pie plot by Evidence Type
    output$etPlot <- renderPlot({
  
        ggplot(et_df, aes(x = factor(1), y = value, fill = Evidence.Type))+
        geom_bar(width = 1, stat = 'identity')+
        coord_polar("y") + 
        scale_fill_brewer(palette="Dark2") +
        blank_theme + 
        xlab("") + 
        ylab("")  +
        theme(axis.text.x=element_blank(),
              axis.text.y=element_blank(),
              legend.position="bottom",
              legend.box = "vertical",
              legend.title = element_blank()) +
        ggtitle("Evidence Loss by Evidence Type")

    })
    # Pie plot by Department
    output$ctPlot <- renderPlot({

        ggplot(ct_df, aes(x = factor(1), y = value, fill = Case.Type))+
        geom_bar(width = 1, stat = 'identity')+
        coord_polar("y") + 
        scale_fill_brewer(palette="Set2") +
        blank_theme +
        xlab("") + 
        ylab("")  +
        theme(axis.text.x=element_blank(),
              axis.text.y=element_blank(),
              legend.position="bottom",
              legend.box = "vertical",
              legend.title = element_blank()) +
        ggtitle("Evidence Loss by Case Type")

    })
    # Pie plot by Case Type
    output$dtPlot <- renderPlot({

        ggplot(dt_df, aes(x = factor(1), y = value, fill = Department))+
        geom_bar(width = 1, stat = 'identity')+
        coord_polar("y") +
        scale_fill_brewer(palette="Paired") +
        blank_theme + 
        xlab("") +
        ylab("") +
        theme(axis.text.x=element_blank(),
              axis.text.y=element_blank(),
              legend.position="bottom",
              legend.box = "vertical",
              legend.title = element_blank()) +
        ggtitle("Evidence Loss by Department")

    })

}

# Run the application 
shinyApp(ui = ui, server = server)



Listening on http://127.0.0.1:3976
`geom_smooth()` using method = 'loess' and formula 'y ~ x'
